# Optimisation solutions
The optimum solution for each week differs. To improve converge quality and speed, the optimum for each week shall be calculated and stored

In [ ]:
# imports
import json
import os

from src.grid import Grid
from src.generators import (
    SolarGenerator,
    WindGenerator,
    NuclearGenerator,
    CoalGenerator,
    GasGenerator,
)
from src.utils import WEEK_MAP

In [ ]:
# initialise dictionary of optimum initialisations
opt_init_weekly = {}

# loop over each week
for week in WEEK_MAP["week"]:
    # initialise grid for current week
    grid = Grid(
        generators={
            "solar": SolarGenerator,
            "wind": WindGenerator,
            "nuclear": NuclearGenerator,
            "gas": GasGenerator,
            "coal": CoalGenerator,
        },
        week=week,
    )

    # save successul optimisations and print unsuccessful ones
    if grid.optimum is not None:
        opt_init_weekly[int(week)] = {
            k: v / grid.demand.mean()
            for k, v in grid.optimum["installed_capacity"].items()
        }
    else:
        print(grid.optimum)

In [ ]:
# save file
with open(os.path.join(os.getcwd(), "weekly_optimum.json"), "w") as f:
    f.write(json.dumps(opt_init_weekly))